# **1. Import Libraries**

In [1]:
import os
import pandas as pd
from pubchempy import get_compounds, NotFoundError, PubChemHTTPError
import re
import time
import requests
from scipy.signal import find_peaks
import numpy as np

# **2. Define Data Directory and Output File**

In [2]:
data_dir = "./../PhotochemCAD/Common Compounds/"
output_file = "./../data/molecular_data.csv"

# **3. Function to Extract Data from .abs.txt Files**

In [3]:
def extract_absorption_data(
    filepath, threshold=0.01, normalize=True, max_peaks=3, min_distance=50
):
    """
    Extracts up to three highest absorption maxima and corresponding wavelengths from a .abs.txt file.
    - Prevents repeated peaks caused by normalization artifacts.
    - Includes handling for empty files to avoid errors.

    Args:
        filepath: Path to the .abs.txt file.
        threshold: Minimum absorbance value for a peak to be considered significant.
        normalize: If True, normalize absorbance values between 0 and 1.
        max_peaks: The maximum number of peaks to extract.
        min_distance: Minimum distance between peaks to prevent closely grouped peaks.

    Returns:
        A list of tuples containing (wavelength, absorption) for the top peaks.
    """
    try:
        with open(filepath, "r") as f:
            lines = f.readlines()

        # Skip header lines
        data_start_index = next(
            (
                i
                for i, line in enumerate(lines)
                if line.strip().startswith("Wavelength")
            ),
            None,
        )

        # If no header is found, return an empty list
        if data_start_index is None:
            print(f"Warning: No data found in {filepath}.")
            return []

        data_start_index += 1  # Move to the data part after the header
        wavelengths = []
        absorptions = []

        for line in lines[data_start_index:]:
            parts = line.strip().split()
            if len(parts) == 2:
                try:
                    wavelengths.append(float(parts[0]))
                    absorptions.append(float(parts[1]))
                except ValueError:
                    print(f"Invalid data format in {filepath}. Skipping line.")
                    continue

        # Convert to numpy arrays
        if not wavelengths or not absorptions:
            print(f"Warning: No valid data found in {filepath}.")
            return []

        wavelengths = np.array(wavelengths)
        absorptions = np.array(absorptions)

        # Check for zero-sized arrays before normalization
        if normalize and len(absorptions) > 0 and absorptions.max() > 0:
            absorptions /= absorptions.max()
        else:
            print(
                f"Warning: Empty or zero max absorption data in {filepath}. Skipping normalization."
            )

        # Identify peaks with separation of 50nm distance
        peaks, _ = find_peaks(absorptions, height=threshold, distance=min_distance)
        peak_data = [(wavelengths[i], absorptions[i]) for i in peaks]

        # Sort by intensity and limit to top N peaks
        peak_data.sort(key=lambda x: x[1], reverse=True)
        peak_data = peak_data[:max_peaks]

        return peak_data

    except (FileNotFoundError, ValueError, IndexError) as e:
        print(f"Error processing {filepath}: {e}")
        return []

# **4. Function to Get SMILES from PubChem**

In [4]:
def get_smiles_from_pubchem(identifier, identifier_type="name"):
    """
    Retrieves SMILES string from PubChem using either name, CAS or CID, with retries.

    Args:
        identifier: Name, CAS, or CID of the molecule.
        identifier_type: Type of identifier ("name", "cas", or "cid").

    Returns:
        SMILES string if found, otherwise None.
    """

    max_retries = 3
    retry_delay = 2

    identifier = identifier.replace("_", " ")

    if identifier_type == "cid":
        if not identifier.isdigit():
            print(
                f"Invalid CID format for identifier: {identifier}. CID must be numeric."
            )
            return None

    for attempt in range(max_retries):
        try:
            if identifier_type == "cid":
                url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{identifier}/property/IsomericSMILES/txt"
                response = requests.get(url)
                if response.status_code == 200:
                    return response.text.strip()
                else:
                    print(
                        f"Failed to retrieve data for CID '{identifier}' with status code: {response.status_code}"
                    )
                    return None
            elif identifier_type == "cas" or identifier_type == "name": 
                url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{identifier}/property/IsomericSMILES/txt"
                response = requests.get(url)
                if response.status_code == 200:
                    return response.text.strip()
                else:
                    print(
                        f"Failed to retrieve data for synonym '{identifier}' with status code: {response.status_code}"
                    )
                    return None
        except NotFoundError:
            return None
        except PubChemHTTPError as e:
            if e.msg == "Status: 503":
                print(
                    f"PubChem service unavailable, retrying in {retry_delay} seconds... (Attempt {attempt + 1})"
                )
                time.sleep(retry_delay)
            else:
                print(
                    f"PubChem HTTP Error for {identifier_type} '{identifier}': {e.msg}"
                )
                return None
        except Exception as e:
            print(
                f"An unexpected error occurred for {identifier_type} '{identifier}': {e}"
            )
            return None

    print(
        f"Failed to retrieve SMILES for {identifier_type} '{identifier}' after multiple attempts."
    )
    return None

# **5. Main Processing Loop**

In [ ]:
data = []
processed_molecules = set()

for filename in os.listdir(data_dir):
    if filename.endswith(".abs.txt"):
        filepath = os.path.join(data_dir, filename)

        # Regex match to extract molecule information
        match = re.match(r"([A-Z]\d+)_((\d+-)+\d+)_(.+?)\.abs\.txt", filename)
        if not match:
            print(f"Skipping file {filename} due to invalid format.")
            continue

        molecule_code, molecule_id, molecule_name = (
            match.group(1),
            match.group(2),
            match.group(4),
        )

        # Prevent processing duplicates
        if molecule_name in processed_molecules:
            continue
        processed_molecules.add(molecule_name)

        # **Attempt to retrieve SMILES using CAS or Name**
        smiles = get_smiles_from_pubchem(molecule_id, "cas")
        if smiles is None:
            print(f"Trying with name for {molecule_name}")
            smiles = get_smiles_from_pubchem(molecule_name, "name")

        # If both CAS and Name searches fail, fallback to placeholder
        if smiles is None:
            print(
                f"Could not retrieve SMILES for {molecule_name}. Skipping the molecule."
            )
            continue

        # Convert SMILES into a list
        smiles_list = [s.strip() for s in smiles.splitlines() if s.strip()]

        # Extract up to 3 peaks using modified extraction function
        peaks = extract_absorption_data(filepath, max_peaks=3)

        # Ensure peaks are valid before iterating
        if peaks and isinstance(peaks, list) and len(peaks) > 0:
            for smile in smiles_list:
                for wavelength, absorption in peaks:
                    data.append(
                        {
                            "Molecule Code": molecule_code,
                            "Molecule CAS": molecule_id,
                            "Molecule Name": molecule_name.replace("_", " "),
                            "SMILES": smile,
                            "Absorption Maxima": absorption,
                            "Wavelength": wavelength,
                        }
                    )
        else:
            # Log for molecules where no valid peaks are detected
            print(f"No valid peaks found in {filename} or error during extraction.")

In [ ]:
df = pd.DataFrame(data)
df.to_csv(output_file, index=False)

print(f"Data saved to {output_file}")